In [1]:
from finance.mongo.extraction import query_mongodb
from finance.params import *
import numpy as np
import pandas as pd


In [2]:
exchange_ls = EXCHANGE_LS
table_name = 'key_ratio'
kwargs = {'calendarYear': { "$gt": 2022 }}

In [3]:
client = PY_MONGO_CLIENT
collection = client['finance'][table_name]

In [25]:
collection.update_many({}, [{'$set': {'calendarYear': {'$toInt': '$calendarYear'}}}])
collection.update_many({}, [{'$set': {'date': {'$toDate': '$date'}}}])

UpdateResult({'n': 3678968, 'nModified': 3678968, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [4]:
result = query_mongodb(exchange_ls=[], table=table_name, **kwargs)

exchange {'calendarYear': {'$gt': 2022}} - 119959


In [5]:
df  = pd.DataFrame(result)

In [6]:
df.shape

(119959, 60)

In [7]:
df = df.drop(columns='_id').drop_duplicates()


In [8]:
df.shape

(59980, 59)

In [9]:
df.dropna(inplace=True)

In [10]:
df_small = df.select_dtypes(include='float64')
df_small

,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,grossProfitMargin,operatingProfitMargin,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
0,0.374705,0.342218,0.029178,76.639837,0.000000,76.639837,816.277407,-739.637569,0.711428,-0.207323,...,3.815997,-4.601518,-20.337391,-21.027670,-21.027670,0.000000,3.815997,0.000000,-29.076732,-1.051950
1,1.642388,1.560930,0.357508,108.026195,0.000000,108.026195,19.716626,88.309569,0.719812,-0.032321,...,10.810373,-124.248378,151.809306,149.686187,149.686187,1.471950,10.810373,0.000690,196.201847,21.580184
2,1.599418,1.429121,0.405268,104.454652,0.000000,104.454652,7.899105,96.555546,0.677672,-0.138433,...,13.725316,-21.866842,94.766608,89.688305,89.688305,-0.176346,13.725316,0.000608,-141.676676,28.156364
3,1.675344,1.519580,0.354381,110.017943,0.000000,110.017943,11.547062,98.470881,0.699102,-0.068227,...,12.304106,-48.192957,70.728398,58.799271,58.799271,0.302105,12.304106,0.000618,-352.927421,25.920684
4,1.291686,0.890847,0.203204,27.188659,14.816474,42.005133,32.853696,9.151437,0.121711,0.073884,...,1.346533,6.363925,29.169728,19.881044,19.881044,0.239772,1.346533,0.008711,20.276735,1.731956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119192,1.757204,0.935524,0.342161,55.243577,165.723455,220.967032,35.122068,185.844964,0.625489,0.135896,...,9.797470,31.250000,189.468882,79.030126,79.030126,0.535714,9.797470,0.000000,60.008008,2.974593
119193,1.732013,0.903313,0.357409,56.276571,183.121711,239.398282,37.210688,202.187594,0.618325,0.086854,...,10.950135,51.041667,-910.227446,358.953660,358.953660,0.000000,10.950135,0.000000,79.623783,3.061897
119363,2.464200,1.028467,0.036953,69.052589,105.994989,175.047578,59.606703,115.440875,0.094153,0.021725,...,0.655435,41.043709,-15.761424,130.941064,130.941064,-0.492525,0.655435,0.000000,19.101905,0.492912
119364,2.445956,0.942038,0.001395,61.115825,108.454797,169.570622,62.286696,107.283926,0.131323,0.056256,...,0.366274,2.226500,27.244150,7.611691,7.611691,0.022265,0.366274,0.000000,10.078868,0.289220


In [11]:
from sklearn.preprocessing import StandardScaler

df_small_ss = StandardScaler().fit_transform(df_small)

In [12]:
from sklearn.decomposition import PCA

In [13]:
pca = PCA(26)
pca.fit(df_small_ss)

PCA(n_components=26)

In [43]:
for i, pc in enumerate(np.cumsum(pca.explained_variance_ratio_)):
    print(i, pc)

0 0.1318615721567029
1 0.2186162500889034
2 0.2921196363767331
3 0.3478155945183306
4 0.4033712898774111
5 0.45883232515762196
6 0.5113467533005942
7 0.5535728076613944
8 0.5917690848475277
9 0.628806124313324
10 0.6644662381334934
11 0.6903403454768522
12 0.7115817926625182
13 0.7324011576377455
14 0.7518050849254537
15 0.7704719307305242
16 0.7890172431527883
17 0.8075426919701587
18 0.8260634654572654
19 0.8445817247998744
20 0.863093307684029
21 0.8816016794422266
22 0.9000687684778816
23 0.9177017556666361
24 0.9350567913584465
25 0.9510394028097319
26 0.9668360694847594
27 0.9801408024245543
28 0.9872978655854014
29 0.9904439970006808
30 0.993485120390619
31 0.9950064404103836
32 0.9964047462646684
33 0.9975503793194537
34 0.9986775163430271
35 0.9995169434362727
36 0.9998398219975719
37 0.9999460823540667
38 0.9999820840788851
39 0.9999934298150556
40 0.9999990215117328
41 0.999999786093233
42 0.9999999961329544
43 0.9999999999846357
44 0.9999999999999999
45 0.9999999999999999
4

In [14]:
pca = PCA(26)
pca.fit(df_small_ss)

PCA(n_components=26)

In [15]:
new = pca.fit_transform(df_small)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP
from hdbscan import HDBSCAN

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine')
new_umap = umap_model.fit_transform(df_small_ss)

In [45]:
hdbscan_model = HDBSCAN(
    min_cluster_size=30,
    metric='euclidean',
    prediction_data=False)

new_cluster = hdbscan_model.fit(new_umap)
df['cluster'] = new_cluster.labels_

In [46]:
df.cluster.value_counts()

cluster
-1      9093
 67     2863
 21     1422
 18      823
 78      751
        ... 
 4        31
 103      31
 145      31
 143      30
 163      30
Name: count, Length: 169, dtype: int64

In [48]:
df[df['symbol']=='NVDA']

,symbol,date,calendarYear,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,...,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,key,cluster
72986,NVDA,2023-10-29,2024,Q3,3.588397,3.528074,0.606417,41.269868,91.125000,132.394868,...,144.010984,138.550666,138.550666,0.549526,56.062554,0.000096,93.194440,18.760683,2024Q3NVDA,55
72987,NVDA,2023-07-30,2024,Q2,2.786627,2.234275,0.559609,47.082254,96.096415,143.178669,...,190.725890,182.042875,182.042875,0.232040,85.556243,0.000086,156.601156,42.020587,2024Q2NVDA,55
72988,NVDA,2023-04-30,2024,Q1,3.427410,2.672176,0.699587,51.056730,163.125000,214.181730,...,268.147578,245.302989,245.302989,1.915659,99.287681,0.000139,271.113577,29.122227,2024Q1NVDA,-1
72989,NVDA,2023-01-29,2023,Q4,3.515618,2.609020,0.516380,56.921170,209.337241,266.258411,...,271.351540,209.938497,209.938497,0.751302,78.028703,0.000208,270.009933,21.363363,2023Q4NVDA,-1
72990,NVDA,2022-10-30,2023,Q3,3.387746,2.633260,0.408461,74.476480,145.555556,220.032035,...,-2428.481957,854.924770,854.924770,32.034534,56.504891,0.000298,317.415600,15.697715,2023Q3NVDA,37
72991,NVDA,2022-07-31,2023,Q2,3.620494,2.951802,0.397861,71.379773,92.375297,163.755070,...,549.704839,362.285787,362.285787,-2.922402,68.631108,0.000217,510.655719,19.290719,2023Q2NVDA,78
72992,NVDA,2022-05-01,2023,Q1,5.317332,4.634304,0.698849,59.051641,99.639482,158.691123,...,357.297066,282.782773,282.782773,-1.650175,59.060929,0.000204,139.693534,18.597910,2023Q1NVDA,-1


In [57]:
df[df['cluster']==55].sort_values('priceToFreeCashFlowsRatio').tail()

,symbol,date,calendarYear,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,...,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,key,cluster
7800,SHOP,2023-03-31,2023,Q1,6.712344,6.532276,1.968290,54.011936,0.0,54.011936,...,711.788997,612.138538,612.138538,-2.042129,40.592741,0.0,-159.776519,7.238247,2023Q1SHOP,55
46765,307.F,2023-03-31,2023,Q1,6.712344,6.532276,1.968290,54.011936,0.0,54.011936,...,717.408853,616.971613,616.971613,-2.058252,40.913237,0.0,-161.045043,7.295396,2023Q1307.F,55
14678,SHOP.TO,2023-06-30,2023,Q2,6.714773,6.546591,1.830682,52.119244,0.0,52.119244,...,842.457744,692.528823,692.528823,0.007739,48.239906,0.0,103.624236,10.949806,2023Q2SHOP.TO,55
46764,307.F,2023-06-30,2023,Q2,6.714773,6.546591,1.830682,52.119244,0.0,52.119244,...,848.177662,697.230790,697.230790,0.007331,48.567434,0.0,104.331930,11.024150,2023Q2307.F,55
7799,SHOP,2023-06-30,2023,Q2,6.714773,6.546591,1.830682,52.119244,0.0,52.119244,...,852.725089,700.968929,700.968929,0.007371,48.827824,0.0,104.894558,11.083255,2023Q2SHOP,55


In [58]:
moats = ['symbol', 'operatingProfitMargin', 'date', 'freeCashFlowOperatingCashFlowRatio', 'netProfitMargin', 'returnOnAssets', 'returnOnEquity']
health = ['currentRatio', 'debtEquityRatio', 'receivablesTurnover']

In [66]:
moat = pd.DataFrame()

moat['symbol'] = df['symbol']
moat['date'] = df['date']
moat['operatingmargin'] = df['operatingProfitMargin'].apply(lambda x: x/0.15)
moat['fcfmargin'] = df['freeCashFlowOperatingCashFlowRatio'].apply(lambda x: x/0.05)
moat['netmargin'] = df['netProfitMargin'].apply(lambda x: x/0.15)
moat['roa'] = df['returnOnAssets'].apply(lambda x: x/0.06)
moat['roe'] = df['returnOnEquity'].apply(lambda x: x/0.01)

In [67]:
df[moats]

,symbol,operatingProfitMargin,date,freeCashFlowOperatingCashFlowRatio,netProfitMargin,returnOnAssets,returnOnEquity
0,MYID.V,-0.207323,2023-03-31,1.033941,-0.207323,-0.195103,0.057152
1,PEGA,-0.032321,2023-09-30,0.986015,-0.021752,-0.005954,-0.043421
2,PEGA,-0.138433,2023-06-30,0.946413,-0.156919,-0.039251,-0.321907
3,PEGA,-0.068227,2023-03-31,0.831339,-0.063827,-0.016206,-0.134463
4,PSX,0.073884,2023-09-30,0.681564,0.052897,0.026959,0.068038
...,...,...,...,...,...,...,...
119192,OSSFF,0.135896,2023-06-30,0.417114,0.078380,0.011711,0.023797
119193,OSSFF,0.086854,2023-03-31,-0.394356,0.053633,0.007272,0.014997
119363,OBGRF,0.021725,2023-03-31,-8.307692,0.003992,0.001405,0.003002
119364,OBGRF,0.056256,2022-12-31,0.279388,0.041127,0.015429,0.032475


In [117]:
ss = StandardScaler()

moat_ss = ss.fit_transform(moat.drop(columns=['symbol', 'date']))

pca = PCA(4)

moat_pca = pca.fit_transform(moat_ss)

pca.explained_variance_ratio_

array([0.39701683, 0.17093579, 0.16674038, 0.16230404])

In [127]:
hdbscan_model = HDBSCAN(
    min_cluster_size=100,
    metric='euclidean',
    prediction_data=False,
    min_samples=15)

new_cluster = hdbscan_model.fit(moat_pca)
moat['cluster'] = new_cluster.labels_

In [128]:
moat['cluster'] .value_counts()

cluster
-1      5082
 88     3320
 43     1450
 3      1340
 13      627
        ... 
 34      105
 95      102
 65      101
 12      101
 113     100
Name: count, Length: 116, dtype: int64

In [129]:
moat[moat['symbol']=='NVDA']

,symbol,date,operatingmargin,fcfmargin,netmargin,roa,roe,cluster
72986,NVDA,2023-10-29,3.832597,19.241680,3.400662,2.844980,17.069883,-1
72987,NVDA,2023-07-30,3.356284,19.089477,3.054219,2.081189,22.501000,108
72988,NVDA,2023-04-30,1.983686,18.296118,1.893771,0.765857,8.331974,90
72989,NVDA,2023-01-29,1.384895,15.473544,1.557869,0.572256,6.397901,90
72990,NVDA,2022-10-30,0.675547,-7.040816,0.764346,0.279918,3.185161,77
72991,NVDA,2022-07-31,0.496221,13.181102,0.652347,0.251480,2.750409,72
72992,NVDA,2022-05-01,1.502574,15.829001,1.301480,0.596449,6.147416,90


In [130]:
moat[moat['cluster']==90].groupby('symbol').count().sort_values('date').tail(40)

,date,operatingmargin,fcfmargin,netmargin,roa,roe,cluster
symbol,,,,,,,
POOL,2,2,2,2,2,2,2
ATLKY,2,2,2,2,2,2,2
ATLFF,2,2,2,2,2,2,2
ATLCY,2,2,2,2,2,2,2
ANET,2,2,2,2,2,2,2
ALD.DE,2,2,2,2,2,2,2
BF-A,2,2,2,2,2,2,2
0Y0Y.L,2,2,2,2,2,2,2
CMC,2,2,2,2,2,2,2


In [62]:
df[moats].netProfitMargin

0        -0.207323
1        -0.021752
2        -0.156919
3        -0.063827
4         0.052897
            ...   
119192    0.078380
119193    0.053633
119363    0.003992
119364    0.041127
119365    0.020230
Name: netProfitMargin, Length: 32610, dtype: float64